### Вспоминаем генераторы

In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    for i in range(2, max_number):
        fib_1, fib_2 = fib_2, fib_1 + fib_2
        yield fib_2

fibs = generate_fib(10)

print(next(fibs))
print(next(fibs))
print(next(fibs))
print(next(fibs))


1
1
2
3


In [ ]:
fibs = generate_fib(10)
for x in fibs:
    print(x)
print('-'*10)
for x in fibs:
    print(x)

1
1
2
3
5
8
13
21
34
55
----------


Кстати, а как вернуть значение из генератора?

In [ ]:
next(fibs)

StopIteration: 

In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    return 4

fibs = generate_fib(10)
for x in fibs:
    print(x)

1
1


In [ ]:
next(fibs)

StopIteration: 

In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    return 4

fibs = generate_fib(10)

try:
    for x in fibs:
        print(x)
except StopIteration as e:
    print("Return value:", e.value)


1
1


In [ ]:
def generate_fib(max_number):
    fib_1, fib_2 = 1, 1
    yield fib_1
    yield fib_2

    return 4

fibs = generate_fib(10)

while True:
    try:
        value = next(fibs)
        print(value)
    except StopIteration as e:
        print("Return value:", e.value)  # Выводим значение return
        break


1
1
Return value: 4


### Генераторы как корутины

Генераторы можно использовать как корутины, которые могут не только возвращать значения, но и принимать их. Это стало основой для использования генераторов в асинхронных сценариях.



In [ ]:
def coroutine():
    print("Start")
    value = yield
    print(f"Received: {value}")
    yield "Done"

coro = coroutine()

next(coro)

coro.send("Hello")


Start
Received: Hello


'Done'

Чуть более сложный пример с циклом внутри

In [ ]:
def running_total():
    total = 0
    while True:
        number = yield total
        if number is None:
            break
        total += number
    return total

gen = running_total()

next(gen)

print(gen.send(5))
print(gen.send(10))
print(gen.send(3))

gen.send(None)


105
115
118


StopIteration: 

В этом примере генератор аккумулирует суммы, принимая данные через `send()`. Такой подход можно использовать для отслеживания состояния в течение выполнения, что напоминает принципы асинхронного программирования, где состояния задач могут изменяться во времени.



Более сложный пример с обработкой исключений в генераторах:

In [ ]:
def exception_handling_coroutine():
    print("Starting coroutine")
    try:
        while True:
            try:
                value = yield
            except ValueError:
                print("ValueError caught inside coroutine!")
            else:
                print(f"Received value: {value}")
    finally:
        print("Coroutine terminating")

coro = exception_handling_coroutine()
next(coro)

coro.send(10)
coro.send(20)

coro.throw(ValueError)

coro.close()


Starting coroutine
Received value: 10
Received value: 20
ValueError caught inside coroutine!
Coroutine terminating


### yield from

Когда `yield from` применяется к подгенератору, он последовательно возвращает все значения этого подгенератора в внешний генератор.


In [ ]:
def accumulator():
    total = 0
    for i in range(3):
        total += i
        yield i
    return total

def main_generator():
    result = yield from accumulator()
    print("Accumulated total:", result)

for value in main_generator():
    print("Yielded:", value)


Yielded: 0
Yielded: 1
Yielded: 2
Accumulated total: 3


Более сложный пример с несколькими подгенераторами

In [ ]:
def numbers():
    yield 1
    yield 2
    yield 3
    return "Numbers done"

def letters():
    yield 'A'
    yield 'B'
    yield 'C'
    return "Letters done"

def main_generator():
    result1 = yield from numbers()
    print("First subgenerator result:", result1)

    result2 = yield from letters()
    print("Second subgenerator result:", result2)

    return "All subgenerators done"

for value in main_generator():
    print("Yielded:", value)


Yielded: 1
Yielded: 2
Yielded: 3
First subgenerator result: Numbers done
Yielded: A
Yielded: B
Yielded: C
Second subgenerator result: Letters done


А теперь давайте перемешаем порядок обращения к подгенераторам!

In [ ]:
def numbers():
    yield 1
    yield 2
    yield 3
    yield 4
    return "Numbers done"

def letters():
    yield 'A'
    yield 'B'
    yield 'C'
    return "Letters done"


def interleaved_generator():
    gens = [numbers(), letters()]
    results = []

    while gens:
        for gen in gens.copy():
            try:
                value = next(gen)
                yield value
            except StopIteration as e:
                results.append(e.value)
                gens.remove(gen)

    for i, result in enumerate(results, start=1):
        print(f"Subgenerator {i} result:", result)

for value in interleaved_generator():
    print("Yielded:", value)


Yielded: 1
Yielded: A
Yielded: 2
Yielded: B
Yielded: 3
Yielded: C
Yielded: 4
Subgenerator 1 result: Letters done
Subgenerator 2 result: Numbers done


In [ ]:
from collections import deque

def numbers():
    yield 1
    yield 2
    yield 3
    yield 4
    return "Numbers done"

def letters():
    yield 'A'
    yield 'B'
    yield 'C'
    return "Letters done"

def interleaved_generator():
    gens = deque([numbers(), letters()])
    results = []

    while gens:
        gen = gens.popleft()
        try:
            value = next(gen)
            yield value
            gens.append(gen)
        except StopIteration as e:
            results.append(e.value)

    for i, result in enumerate(results, start=1):
        print(f"Subgenerator {i} result:", result)

for value in interleaved_generator():
    print("Yielded:", value)


Yielded: 1
Yielded: A
Yielded: 2
Yielded: B
Yielded: 3
Yielded: C
Yielded: 4
Subgenerator 1 result: Letters done
Subgenerator 2 result: Numbers done
